<a href="https://colab.research.google.com/github/SananSuleymanov/Neural_Style_Transfer_video/blob/main/NeuralStyle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Steps to follow:**
1. Load imports
2. Load models
2. Read style image and preprocess
3. Read the frame of the video and preprocess it
4. Apply Neural style transfer to the frame 
5. Fill in the gaps

#Install / load deps

In [11]:
#Uncomment this to install these packages

# import sys
# !pip install tensorflow_hub
# !pip install tensorflow
# !pip install matplotlib

import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv2
import PIL
import asyncio



#Load models

In [6]:
hub_model = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')
filmModel = hub.load('https://tfhub.dev/google/film/1')


#Processing image

In [7]:
#read image, convert to tensor, normalize and resize 
def image_read(image):
  max_dim=512
  image= tf.convert_to_tensor(image, dtype = tf.float32)
  image= image/255.0
  shape = tf.cast(tf.shape(image)[:-1], tf.float32)
  long_dim = max(shape)
  scale = max_dim/long_dim
  new_shape = tf.cast(shape*scale, tf.int32)
  new_image = tf.image.resize(image, new_shape)
  new_image = new_image[tf.newaxis, :]
  
  return new_image

In [8]:
#convert tensor to numpy array
def tensor_toimage(tensor):
  tensor =tensor*255
  tensor = np.array(tensor, dtype=np.uint8)
  if np.ndim(tensor)>3:
    assert tensor.shape[0]==1
    tensor=tensor[0]

  return tensor

In [75]:
async def interpolated_frames(pim_image_numpy, nim_image_numpy, i):
    time = np.array([i], dtype=np.float32)
    input = {
    'x0': np.expand_dims(pim_image_numpy, axis=0), # adding the batch dimension to the time
    'x1': np.expand_dims(nim_image_numpy, axis=0),
    'time': np.expand_dims(time, axis=0)
    }
    return filmModel(input)['image']


#Transfer Video

choose your video and style here...



In [87]:
style_im = cv2.imread("./style6.webp")
style_im = cv2.cvtColor(style_im, cv2.COLOR_BGR2RGB)
style_im = image_read(style_im)
cap = cv2.VideoCapture("./content1.m4v")

#in order to get the size of width and shape of video, we used first frame of video
ret, frame = cap.read()

frame_width = image_read(frame)[0].shape[1]
frame_height= image_read(frame)[0].shape[0]


out = cv2.VideoWriter('output.mp4', cv2.VideoWriter_fourcc(*'XVID'),20, 
                      (frame_width,frame_height))


# resue the generated image as a base for the next frame?
prev_image = None
count = 0
while True:
  ret, frame = cap.read()

  # if count > 15:
  #   break
  if ret == True:
    if count % 10 == 0:
      frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
      frame = image_read(frame)
      stylized_frame = hub_model(tf.constant(frame), tf.constant(style_im))[0]
      next_image = tensor_toimage(stylized_frame)

      if(prev_image is not None):
        pim_image_numpy = tf.cast(prev_image, dtype=tf.float32).numpy()/ 255
        nim_image_numpy = tf.cast(next_image, dtype=tf.float32).numpy()/ 255
        # generate images in the middle
        L = await asyncio.gather(
          interpolated_frames(pim_image_numpy, nim_image_numpy, 0.2),
          interpolated_frames(pim_image_numpy, nim_image_numpy, 0.4),
          interpolated_frames(pim_image_numpy, nim_image_numpy, 0.6),
          interpolated_frames(pim_image_numpy, nim_image_numpy, 0.8)
        )
        for f in L:
          out.write(tensor_toimage(f))
        print("frame is written", count)
      prev_image = next_image
      
      # replace style image with the generated image
      # style_im = stylized_frame

      # generate frames in the middle...
      
      out.write(next_image)
      print("frame is written", count)
  else:
    break
  count+=1

cap.release()
out.release()

frame is written 0
frame is written 10
frame is written 10
frame is written 20
frame is written 20
frame is written 30
frame is written 30
frame is written 40
frame is written 40
frame is written 50
frame is written 50
frame is written 60
frame is written 60
frame is written 70
frame is written 70
frame is written 80
frame is written 80
frame is written 90
frame is written 90
frame is written 100
frame is written 100
frame is written 110
frame is written 110
frame is written 120
frame is written 120
frame is written 130
frame is written 130
frame is written 140
frame is written 140
frame is written 150
frame is written 150
frame is written 160
frame is written 160
frame is written 170
frame is written 170
frame is written 180
frame is written 180
frame is written 190
frame is written 190
frame is written 200
frame is written 200
frame is written 210
frame is written 210
frame is written 220
frame is written 220
frame is written 230
frame is written 230
frame is written 240
frame is wri